Pandas makes it easy to import data from csv files. But it can also read data from a variety of formats, including Excel, SAS, SPSS, and others.

In this notebook, you will see how to import data from an Excel file.

In [1]:
import pandas as pd

### Option 1: Read in with a MultiIndex


We'll be importing from the file `2018 County Health Rankings Data - v2.xls`. 

When reading in an Excel file, you can specify which sheet you want to read in. Remember when using this argument that Python uses zero-based indexing. In this example, we'll import the data from the `Outcomes & Factors Rankings` sheet. To point `pandas` to the correct sheet, we can use `sheet_name = 1`.

If you inspect this file in Excel, you will see that it has a header on rows 0 and 1. We need to let `pandas` know this, and we can do so by using the `header` argument and passing in the list `[0,1]`.

In [2]:
health_outcomes = pd.read_excel('../data/2018 County Health Rankings Data - v2.xls', sheet_name = 1, header = [0,1])

Take a look at what was read in.

In [3]:
health_outcomes.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
                FIPS              State             County   
0               1001            Alabama            Autauga   
1               1003            Alabama            Baldwin   
2               1005            Alabama            Barbour   
3               1007            Alabama               Bibb   
4               1009            Alabama             Blount   

    Unnamed: 3_level_0 Health Outcomes          Health Factors           
  # of Ranked Counties            Rank Quartile           Rank Quartile  
0                   67              11        1              8        1  
1                   67               3        1              3        1  
2                   67              34        2             56        4  
3                   67              41        3             37        3  
4                   67              14        1             19        2

You may notice that the column names look strange. If you check the `.columns` attribute, you'll see what happened.

In [4]:
health_outcomes.columns

MultiIndex([('Unnamed: 0_level_0',                 'FIPS'),
            ('Unnamed: 1_level_0',                'State'),
            ('Unnamed: 2_level_0',               'County'),
            ('Unnamed: 3_level_0', '# of Ranked Counties'),
            (   'Health Outcomes',                 'Rank'),
            (   'Health Outcomes',             'Quartile'),
            (    'Health Factors',                 'Rank'),
            (    'Health Factors',             'Quartile')],
           )

Notice that since there were two rows of header, we end up with a MultiIndex.

While you could leave your DataFrame with a MultiIndex, it will probably make it much easier if you collapse it down to a regular index. The next cell provides code for doing that.

In [5]:
# First, combine the two column name levels
col_names = [' '.join(col).strip() for col in health_outcomes.columns.values]

# Then get rid of the "Unnamed" portion for the first 4 columns
col_names[0:4] = ['FIPS', 'State', 'County', '# of Ranked Counties']

Here is the result of the above cell.

In [6]:
col_names

['FIPS',
 'State',
 'County',
 '# of Ranked Counties',
 'Health Outcomes Rank',
 'Health Outcomes Quartile',
 'Health Factors Rank',
 'Health Factors Quartile']

Now, you can assign this new list to be the column names of your DataFrame.

In [7]:
health_outcomes.columns = col_names

In [8]:
health_outcomes.head()

,FIPS,State,County,# of Ranked Counties,Health Outcomes Rank,Health Outcomes Quartile,Health Factors Rank,Health Factors Quartile
0,1001,Alabama,Autauga,67,11,1,8,1
1,1003,Alabama,Baldwin,67,3,1,3,1
2,1005,Alabama,Barbour,67,34,2,56,4
3,1007,Alabama,Bibb,67,41,3,37,3
4,1009,Alabama,Blount,67,14,1,19,2


If we want to merge this data with out other data, we need to conver the state names to a state abbreviation.

In [9]:
state_abbrev=pd.read_csv('../data/state_abbrev.csv')

In [10]:
state_abbrev.head()

,name,abbrev
0,ALABAMA,AL
1,ALASKA,AK
2,ARIZONA,AZ
3,ARKANSAS,AR
4,CALIFORNIA,CA


Since the state names in our abbreviations dataset are uppercase, you will need to convert the state names from the `health_outcomes` DataFrame to uppercase as well before applying `.map`.

In [11]:
health_outcomes['State']=health_outcomes.State.str.upper().map(state_abbrev.set_index('name')['abbrev'].to_dict())

In [12]:
health_outcomes.head()

,FIPS,State,County,# of Ranked Counties,Health Outcomes Rank,Health Outcomes Quartile,Health Factors Rank,Health Factors Quartile
0,1001,AL,Autauga,67,11,1,8,1
1,1003,AL,Baldwin,67,3,1,3,1
2,1005,AL,Barbour,67,34,2,56,4
3,1007,AL,Bibb,67,41,3,37,3
4,1009,AL,Blount,67,14,1,19,2


### Option 2: Read in Specific Columns

Another option you have is to only read in specific columns. Let's say we want to only import the % smokers, which is column AE in the Ranked Measures Data sheet. In this case, to avoid having to deal with a MultiIndex, we'll set `header = 1` to ignore the top row of the Excel sheet.

To point `pandas` to a specific set of columns, you can use the `usecols` argument. You can pass a string which specifies the columns you want, using a range of columns, or comma-separated list of columns. Here, we'll read in columns A, B, C, and AE, which we specify as "A:C,AE".

In [13]:
smokers = pd.read_excel('../data/2018 County Health Rankings Data - v2.xls', sheet_name = 3, header = 1, usecols = 'A:C,AE')

Verify that it read in as expected.

In [14]:
smokers.head()

,FIPS,State,County,% Smokers
0,1001.0,Alabama,Autauga,19.124658
1,1003.0,Alabama,Baldwin,16.795485
2,1005.0,Alabama,Barbour,21.540878
3,1007.0,Alabama,Bibb,19.916404
4,1009.0,Alabama,Blount,19.652158
